In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests
import urllib, json
from PIL import Image
import datetime

In [96]:
# Подключаем api_key из файла
f = open('API_KEY.txt', 'r')
api_key = f.read()
f.close()

In [97]:
# Составляем URL request
api_root = 'https://api.nasa.gov/'
api_product = 'neo/rest/v1/feed?'
start_date = '2020-07-07'
end_date = '2020-07-07'
url = f"{api_root}{api_product}start_date={start_date}&end_date={end_date}&api_key={api_key}"

In [98]:
'''
Функция навигации по данным из api response   
возвращает ссылку на данные в другом временном отрезке

    step = 'next' -- следующий день от заданного start_date в URL request
    step = 'prev' -- предыдущий день от заданного start_date в URL request
    step = 'self' -- текущий день, равный start_date в URL request
'''

def get_url(url, step):
    url_level = urllib.request.urlopen(url)
    data_level = json.loads(url_level.read())['links']
    if step == 'next':
        return data_level[step]
    elif step == 'prev':
        return data_level[step]
    elif step == 'self':
        return data_level[step]

In [99]:
'''
Вариант использования бибилотеки datetime для скольжения по датам (не используется в коде)
и вывод даты в строку

delta = datetime.timedelta(days=1)
day_current = datetime.date.fromisoformat(start_date) + delta
date_str = datetime.date.strftime(day_current, '%Y-%m-%d')
'''

"\nВариант использования бибилотеки datetime для скольжения по датам (не используется в коде)\nи вывод даты в строку\n\ndelta = datetime.timedelta(days=1)\nday_current = datetime.date.fromisoformat(start_date) + delta\ndate_str = datetime.date.strftime(day_current, '%Y-%m-%d')\n"

In [100]:
'''Навигация по ссылкам.

Работаем с данными за один день 2020-07-07.
Обрабатываем входящий json.
'''

url_level_0 = urllib.request.urlopen(url)
data_level_0 = pd.DataFrame.from_dict(json.loads(url_level_0.read()))
data_level_0

,links,element_count,near_earth_objects
next,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,11,NaN
prev,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,11,NaN
self,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,11,NaN
2020-07-07,NaN,11,[{'links': {'self': 'http://api.nasa.gov/neo/r...


In [101]:
# Выбираем один элемент из списка за один день
data_day = data_level_0.near_earth_objects[3]
data_day_item = data_day[0]
data_day_item

{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/3343102?api_key=8Mbg6rxSagXNYxrUbC8ynidJUcLn9h4CC7ucuT21'},
 'id': '3343102',
 'neo_reference_id': '3343102',
 'name': '(2006 SS19)',
 'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3343102',
 'absolute_magnitude_h': 18.9,
 'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.4411182,
   'estimated_diameter_max': 0.9863702813},
  'meters': {'estimated_diameter_min': 441.1181999969,
   'estimated_diameter_max': 986.3702813054},
  'miles': {'estimated_diameter_min': 0.2740980571,
   'estimated_diameter_max': 0.6129018881},
  'feet': {'estimated_diameter_min': 1447.2382352778,
   'estimated_diameter_max': 3236.1230737181}},
 'is_potentially_hazardous_asteroid': False,
 'close_approach_data': [{'close_approach_date': '2020-07-07',
   'close_approach_date_full': '2020-Jul-07 08:13',
   'epoch_date_close_approach': 1594109580000,
   'relative_velocity': {'kilometers_per_second': '11.947301649',
    'kilometers_per_h

In [102]:
# Создаем словари с необходимями данными для DataFrame
dict_estimated_diameter = data_day_item['estimated_diameter']['kilometers']
dict_close_approach_data = data_day_item['close_approach_data'][0]
dict_relative_velocity = dict_close_approach_data['relative_velocity']
dict_miss_distance = dict_close_approach_data['miss_distance']

In [103]:
# Чистим словарь от вложенных словарей
keys = ['links','estimated_diameter','close_approach_data', 'relative_velocity', 'miss_distance']

for key in keys:
    data_day_item.pop(key, None)
data_day_item_short = data_day_item

data_day_item_short

{'id': '3343102',
 'neo_reference_id': '3343102',
 'name': '(2006 SS19)',
 'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3343102',
 'absolute_magnitude_h': 18.9,
 'is_potentially_hazardous_asteroid': False,
 'is_sentry_object': False}

In [104]:
'''Создаем DataFrame для выбранных данных.

Объединяем DataFrame в один для дальнейшей работы.
'''

df1 = pd.DataFrame([data_day_item_short])
df2 = pd.DataFrame(dict_estimated_diameter, index=[0]).reset_index(drop=True)
df3 = pd.DataFrame(dict_close_approach_data, index=[0]).reset_index(drop=True)
df4 = pd.DataFrame(dict_relative_velocity, index=[0]).reset_index(drop=True)
df5 = pd.DataFrame(dict_miss_distance, index=[0]).reset_index(drop=True)
df_item = df1.join(df2).join(df3).join(df4).join(df5)

df_item

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_min,estimated_diameter_max,close_approach_date,...,relative_velocity,miss_distance,orbiting_body,kilometers_per_second,kilometers_per_hour,miles_per_hour,astronomical,lunar,kilometers,miles
0,3343102,3343102,(2006 SS19),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3343102,18.9,False,False,0.441118,0.98637,2020-07-07,...,NaN,NaN,Earth,11.947301649,43010.285936445,26724.9190586761,0.4460874678,173.5280249742,66733735.016573586,41466420.1158480468


In [105]:
# Чистим DataFrame от нерелевантных данных, переименовываем
df_item = df1.join(df2).join(df3).join(df4).join(df5).drop(['nasa_jpl_url',
                                                'neo_reference_id',
                                                'miss_distance',
                                                'miles_per_hour',
                                                'astronomical',
                                                'lunar',
                                                'miles'
                                                ], axis=1
).rename(columns={'kilometers': 'miss_distance_kilometers',
                'estimated_diameter_min': 'estimated_diameter_km_min',
                'estimated_diameter_max': 'estimated_diameter_km_max'})

# Создаем словарь из DataFrame
df_item.to_dict()

{'id': {0: '3343102'},
 'name': {0: '(2006 SS19)'},
 'absolute_magnitude_h': {0: 18.9},
 'is_potentially_hazardous_asteroid': {0: False},
 'is_sentry_object': {0: False},
 'estimated_diameter_km_min': {0: 0.4411182},
 'estimated_diameter_km_max': {0: 0.9863702813},
 'close_approach_date': {0: '2020-07-07'},
 'close_approach_date_full': {0: '2020-Jul-07 08:13'},
 'epoch_date_close_approach': {0: 1594109580000},
 'relative_velocity': {0: nan},
 'orbiting_body': {0: 'Earth'},
 'kilometers_per_second': {0: '11.947301649'},
 'kilometers_per_hour': {0: '43010.285936445'},
 'miss_distance_kilometers': {0: '66733735.016573586'}}

**Создаем цикл для работы с API с получением данных за определенное количество дней**

In [106]:
'''Структуры для работы.

items_list - список из данных по каждому объекту за определенный период
missed_colums_keys = [] - список для обработки словаря по одному объекту (удаление столбцов)
'''
items_list = []
missed_colums_keys = [
    'links','estimated_diameter',
    'close_approach_data',
    'relative_velocity',
    'miss_distance'
    ]

In [107]:
'''Создание списка заголовков

для требуемого пустого DataFrame
из DataFrame за один день
'''
key_list = list(df_item.to_dict())
key_list

['id',
 'name',
 'absolute_magnitude_h',
 'is_potentially_hazardous_asteroid',
 'is_sentry_object',
 'estimated_diameter_km_min',
 'estimated_diameter_km_max',
 'close_approach_date',
 'close_approach_date_full',
 'epoch_date_close_approach',
 'relative_velocity',
 'orbiting_body',
 'kilometers_per_second',
 'kilometers_per_hour',
 'miss_distance_kilometers']

In [108]:
# Создаем пустой DataFrame
df0 = pd.DataFrame(columns=key_list)

df0

,id,name,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_km_min,estimated_diameter_km_max,close_approach_date,close_approach_date_full,epoch_date_close_approach,relative_velocity,orbiting_body,kilometers_per_second,kilometers_per_hour,miss_distance_kilometers


In [109]:
# n - количество запросов(дней)
n = 3

In [112]:
''' Создаем цикл для получения данных

через API за заданный период времени
на основе алгоритма получения данных
за один день
'''


for i in range(1, n+1):
    current = get_url(url, 'next')
    url = current
    
    # Навигация по ссылкам, работаем с данными за один день
    url_level_0 = urllib.request.urlopen(url)
    data_level_0 = pd.DataFrame.from_dict(json.loads(url_level_0.read()))
    
    # Cписок элементов за один день
    data_in_day = data_level_0.near_earth_objects[3]

    # Цикл по элементам списка за один день
    for data_day_item in data_in_day:
        dict_estimated_diameter = data_day_item['estimated_diameter']['kilometers']
        dict_close_approach_data = data_day_item['close_approach_data'][0]
        dict_relative_velocity = dict_close_approach_data['relative_velocity']
        dict_miss_distance = dict_close_approach_data['miss_distance']

        # Чистим словарь от вложенных словарей
        for key in missed_colums_keys:
            data_day_item.pop(key, None)

        data_day_item_short = data_day_item
        
        # Создаем Dataframe из вложенных словарей
        df1 = pd.DataFrame([data_day_item_short])
        df2 = pd.DataFrame(dict_estimated_diameter, index=[0]).reset_index(drop=True)
        df3 = pd.DataFrame(dict_close_approach_data, index=[0]).reset_index(drop=True)
        df4 = pd.DataFrame(dict_relative_velocity, index=[0]).reset_index(drop=True)
        df5 = pd.DataFrame(dict_miss_distance, index=[0]).reset_index(drop=True)
        
        # Чистим DataFrame от нерелевантных данных, переименовываем
        df_item = df1.join(df2).join(df3).join(df4).join(df5).drop(['nasa_jpl_url',
                                                        'neo_reference_id',
                                                        'miss_distance',
                                                        'miles_per_hour',
                                                        'astronomical',
                                                        'lunar',
                                                        'miles'
                                                        ], axis=1
        ).rename(columns={'kilometers': 'miss_distance_kilometers',
                        'estimated_diameter_min': 'estimated_diameter_km_min',
                        'estimated_diameter_max': 'estimated_diameter_km_max'})
        

        # Добавление в итоговый DataFrame данных построчно
        df0 = pd.concat([df0, df_item])

In [111]:
# Итоговый DataFrame.

df0 = df0.head(15).reset_index()

df0

,id,name,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_km_min,estimated_diameter_km_max,close_approach_date,close_approach_date_full,epoch_date_close_approach,relative_velocity,orbiting_body,kilometers_per_second,kilometers_per_hour,miss_distance_kilometers
0,2242450,242450 (2004 QY2),14.55,True,False,3.270054,7.312064,2020-07-08,2020-Jul-08 11:27,1594207620000,NaN,Earth,26.6793702848,96045.7330254319,24410484.006243386
1,2498066,498066 (2007 RM133),18.23,False,False,0.600558,1.342889,2020-07-08,2020-Jul-08 11:04,1594206240000,NaN,Earth,6.418989474,23108.3621065377,33082442.643096557
2,2539063,539063 (2016 MK1),19.12,True,False,0.398616,0.891333,2020-07-08,2020-Jul-08 10:39,1594204740000,NaN,Earth,37.2536882655,134113.2777557244,71644581.721980222
3,3414391,(2008 MG1),19.90,False,False,0.278327,0.622358,2020-07-08,2020-Jul-08 20:04,1594238640000,NaN,Earth,41.2049517212,148337.8261962257,73123553.029094384
4,3542555,(2010 PQ10),21.70,False,False,0.121494,0.271669,2020-07-08,2020-Jul-08 17:39,1594229940000,NaN,Earth,14.4443075061,51999.5070217948,28113262.776487424
5,3763274,(2016 VB1),28.57,False,False,0.005135,0.011483,2020-07-08,2020-Jul-08 03:16,1594178160000,NaN,Earth,10.8587965794,39091.6676858994,60877137.15933323
6,3766298,(2017 AJ3),24.30,False,False,0.036691,0.082043,2020-07-08,2020-Jul-08 01:55,1594173300000,NaN,Earth,13.1892893681,47481.4417252928,58666231.824340652
7,3773593,(2017 FM127),25.60,False,False,0.020163,0.045086,2020-07-08,2020-Jul-08 00:14,1594167240000,NaN,Earth,16.2362229805,58450.4027298886,63281319.621293094
8,3989259,(2020 BL11),27.80,False,False,0.007321,0.016370,2020-07-08,2020-Jul-08 17:29,1594229340000,NaN,Earth,3.4813621372,12532.9036938844,12376593.138754985
9,54017189,(2020 KM3),22.10,False,False,0.101054,0.225964,2020-07-08,2020-Jul-08 05:23,1594185780000,NaN,Earth,7.3867377897,26592.2560429277,20448546.344027874
